# RAND dataset analysis

## Open task on the RAND Terrorism Dataset

This is an open challenge to apply what you have learnt analysing Pride and Prejudice with spaCy on a dataset of real events. We have preprocessed the [RAND Terrorism Dataset]((http://www.rand.org/nsrd/projects/terrorism-incidents.html) for this task reducing the data to 10033 articles from 1968 to 2009.

Can you find out the following using the code you have written?
- Who are the terrorist groups and other persons mentioned in each article?
- What locations are mentioned in each article? Hint: a location just has a different label to a person
- From all of your entities, can you find out which named entities are terrorists from the syntactic relationships?
- With all of this information, can you plot a figure expressing the relationships between locations and terrorists?

There are no right answers to any of these questions, and there might not even be an answer at all.

In [ ]:
# To get you started we can import Pandas and Seaborn which might help you
# build a graph or visualisation of the data

import pandas as pd
import seaborn as sns
import spacy

nlp = spacy.load('en')

def read_file(file_name):
    with open(file_name, 'r') as file:
        return file.read().decode('utf-8')

terrorism_file = read_file('data/rand-terrorism-dataset.txt')
terrorism_articles = nlp(terrorism_file)

#### Example output using Pandas and Seaborn

![Heatmap of terrorist group and country](images/example_output.png)